In [1]:
#  dugoutdetours, 
wikilink ='https://en.wikipedia.org/wiki/List_of_Major_League_Baseball_team_rosters'

In [2]:

# Baseball

import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle



if False:
    # Step 1: Fetch the webpage
    url = wikilink
    response = requests.get(url)
    html_content = response.content
    
    # Step 2: Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Step 3: Find all team sections
    teams = soup.find_all('table', class_='toccolours')
    
    # Initialize lists to store data
    team_names = []
    player_names = []
    player_links = []

    # Step 4: Extract team names, player names, and links
    for team in teams:
        team_name = team.find('div', class_='navbar-header').get_text(strip=True)
        team_name = team_name.split('roster')[0].strip()
    
        # Extract player names and links
        for player in team.find_all('a', href=True, title=True):
            player_name = player.get_text(strip=True)
            player_link = "https://en.wikipedia.org" + player['href']
            team_names.append(team_name)
            player_names.append(player_name)
            player_links.append(player_link)
    
    # Step 5: Create a DataFrame
    data = {
        'Team': team_names,
        'Name': player_names,
        'Name_Link': player_links
    }
    df = pd.DataFrame(data)

    # Filter the DataFrame
    df = df[
        (df['Name'].str.len() >= 6) &
        (~df['Name'].str.contains('injured', case=False)) &
        (~df['Name'].str.contains('personal', case=False))
    ]

    # Pickle the DataFrame
    pickle.dump(df, open('pkls/dugoutdf.pkl', 'wb'))
    
else:
    
    # Load the pickled DataFrame
    df = pickle.load(open('pkls/dugoutdf.pkl', 'rb'))


# Display the DataFrame
df.head()

,Team,Name,Name_Link,birthplace,birthplace_link,photo_link,summary,birth_coords
0,Arizona Diamondbacks,Yilber Díaz,https://en.wikipedia.org/wiki/Yilber_D%C3%ADaz,Guatire,https://en.wikipedia.org/wiki/Guatire,None,Yilber Daniel Díaz is a Venezuelan professiona...,"(10.47166667, -66.54055556)"
1,Arizona Diamondbacks,Zac Gallen,https://en.wikipedia.org/wiki/Zac_Gallen,"Somerdale, New Jersey","https://en.wikipedia.org/wiki/Somerdale,_New_J...",https://upload.wikimedia.org/wikipedia/commons...,Zachary Peter Gallen is an American profession...,"(39.845416, -75.021701)"
2,Arizona Diamondbacks,Ryne Nelson,https://en.wikipedia.org/wiki/Ryne_Nelson,"Henderson, Nevada","https://en.wikipedia.org/wiki/Henderson,_Nevada",None,Ryne Tanner Nelson is an American professional...,"(36.0292, -115.0253)"
3,Arizona Diamondbacks,Brandon Pfaadt,https://en.wikipedia.org/wiki/Brandon_Pfaadt,"Louisville, Kentucky","https://en.wikipedia.org/wiki/Louisville,_Kent...",None,Brandon Connor Pfaadt is an American professio...,"(38.25611111, -85.75138889)"
4,Arizona Diamondbacks,Humberto Castellanos,https://en.wikipedia.org/wiki/Humberto_Castell...,Tepatitlán,https://en.wikipedia.org/wiki/Tepatitl%C3%A1n,https://upload.wikimedia.org/wikipedia/commons...,Humberto Castellanos is a Mexican professional...,"(20.81388889, -102.76388889)"


In [3]:

# Group by 'Team' column
grouped = df.groupby('Team')

# Define a function to filter first 40 players
def filter_first_40(group):
    return group.head(40)

# Apply the filtering function to each group and concatenate the results
filtered_df = grouped.apply(filter_first_40).reset_index(drop=True)

# works!

df = filtered_df

df = df.dropna(subset=['Name', 'Name_Link'])

C:\Users\efrie\AppData\Local\Temp\ipykernel_19512\2903108660.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = grouped.apply(filter_first_40).reset_index(drop=True)


In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import pickle

if False:

    def get_infobox_details(url):
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
    
        # Extract infobox
        infobox = soup.find('table', class_='infobox ib-baseball-bio vcard')
    
        if not infobox:
            return None, None, None
    
        # Initialize default values
        birthplace = None
        birthplace_link = None
        photo_link = None
    
        # Extract the birthplace and link
        birth_row = infobox.find('b', text='Born:')
        if birth_row:
            birth_info = birth_row.find_parent('td').find_next_sibling('td')
            if not birth_info:
                birth_info = birth_row.find_parent('td')
            birthplace_anchor = birth_info.find('a', href=True)
            if birthplace_anchor:
                birthplace = birthplace_anchor.text
                birthplace_link = 'https://en.wikipedia.org' + birthplace_anchor['href']
    
        # Extract the photo link directly from the infobox
        img_tag = infobox.find('img')
        if img_tag:
            img_src = img_tag.get('src')
            if img_src:
                photo_link = 'https:' + img_src
    
        return birthplace, birthplace_link, photo_link
    
    # Assuming df is your original DataFrame
    # Make a new copy of the DataFrame to work with
    df_copy = df.copy()
    
    # Add new columns for birthplace, birthplace_link, and photo_link to the copy
    df_copy['birthplace'] = None
    df_copy['birthplace_link'] = None
    df_copy['photo_link'] = None
    
    # Iterate over the copied dataframe and fetch details
    for index, row in df_copy.iterrows():
        birthplace, birthplace_link, photo_link = get_infobox_details(row['Name_Link'])
        df_copy.at[index, 'birthplace'] = birthplace
        df_copy.at[index, 'birthplace_link'] = birthplace_link
        df_copy.at[index, 'photo_link'] = photo_link
    
        # Add a delay of 0.3 seconds between requests to comply with Wikipedia guidelines
        time.sleep(0.3)
    
    # Pickle the DataFrame
    pickle.dump(df, open('pkls/dugoutdf.pkl', 'wb'))
    
else:
    
    # Load the pickled DataFrame
    df = pickle.load(open('pkls/dugoutdf.pkl', 'rb'))

In [5]:
# get summary

import pandas as pd
import requests
import time

if False:

  # Function to fetch Wikipedia page summary
  def fetch_wikipedia_summary(link):
      time.sleep(0.2)  # Add a delay of 0.2 seconds between requests
      try:
          # Extract page title from the link
          title = link.split('/')[-1]
          url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
          response = requests.get(url)
          data = response.json()
          if 'extract' in data:
              return data['extract']
      except Exception as e:
          print(f"Error fetching summary for {link}: {e}")
      return None

  # Apply the function to each row in the DataFrame
  df['summary'] = df['Name_Link'].apply(fetch_wikipedia_summary)


  # Pickle the DataFrame
  pickle.dump(df, open('pkls/dugoutdf.pkl', 'wb'))

else:

  # Load the pickled DataFrame
  df = pickle.load(open('pkls/dugoutdf.pkl', 'rb'))

df.head()



,Team,Name,Name_Link,birthplace,birthplace_link,photo_link,summary,birth_coords
0,Arizona Diamondbacks,Yilber Díaz,https://en.wikipedia.org/wiki/Yilber_D%C3%ADaz,Guatire,https://en.wikipedia.org/wiki/Guatire,None,Yilber Daniel Díaz is a Venezuelan professiona...,"(10.47166667, -66.54055556)"
1,Arizona Diamondbacks,Zac Gallen,https://en.wikipedia.org/wiki/Zac_Gallen,"Somerdale, New Jersey","https://en.wikipedia.org/wiki/Somerdale,_New_J...",https://upload.wikimedia.org/wikipedia/commons...,Zachary Peter Gallen is an American profession...,"(39.845416, -75.021701)"
2,Arizona Diamondbacks,Ryne Nelson,https://en.wikipedia.org/wiki/Ryne_Nelson,"Henderson, Nevada","https://en.wikipedia.org/wiki/Henderson,_Nevada",None,Ryne Tanner Nelson is an American professional...,"(36.0292, -115.0253)"
3,Arizona Diamondbacks,Brandon Pfaadt,https://en.wikipedia.org/wiki/Brandon_Pfaadt,"Louisville, Kentucky","https://en.wikipedia.org/wiki/Louisville,_Kent...",None,Brandon Connor Pfaadt is an American professio...,"(38.25611111, -85.75138889)"
4,Arizona Diamondbacks,Humberto Castellanos,https://en.wikipedia.org/wiki/Humberto_Castell...,Tepatitlán,https://en.wikipedia.org/wiki/Tepatitl%C3%A1n,https://upload.wikimedia.org/wikipedia/commons...,Humberto Castellanos is a Mexican professional...,"(20.81388889, -102.76388889)"


In [6]:
# get coordinates

import pandas as pd
import requests
import time
import pickle


if False:


  # Function to fetch coordinates from Wikipedia API with error handling
  def fetch_coordinates_from_wikipedia(link):
      try:
          # Extract page title from the link
          title = link.split('/')[-1]
          url = f"https://en.wikipedia.org/w/api.php?action=query&prop=coordinates&format=json&titles={title}"
          response = requests.get(url)
          data = response.json()
          pages = data.get('query', {}).get('pages', {})
          if pages:
              page = next(iter(pages.values()))  # Get the first page (should be only one)
              coords = page.get('coordinates', [{}])[0]
              if coords:
                  return (coords.get('lat'), coords.get('lon'))
          # If there are no pages or coordinates, return None
          return None
      except Exception as e:
          #print(f"Error fetching coordinates for {link}: {e}")
          return None



  df['birth_coords'] = df['birthplace_link'].apply(fetch_coordinates_from_wikipedia)


  # Pickle the DataFrame
  pickle.dump(df, open('pkls/dugoutdf.pkl', 'wb'))

else:

  # Load the pickled DataFrame
  df = pickle.load(open('pkls/dugoutdf.pkl', 'rb'))

df.head()

,Team,Name,Name_Link,birthplace,birthplace_link,photo_link,summary,birth_coords
0,Arizona Diamondbacks,Yilber Díaz,https://en.wikipedia.org/wiki/Yilber_D%C3%ADaz,Guatire,https://en.wikipedia.org/wiki/Guatire,None,Yilber Daniel Díaz is a Venezuelan professiona...,"(10.47166667, -66.54055556)"
1,Arizona Diamondbacks,Zac Gallen,https://en.wikipedia.org/wiki/Zac_Gallen,"Somerdale, New Jersey","https://en.wikipedia.org/wiki/Somerdale,_New_J...",https://upload.wikimedia.org/wikipedia/commons...,Zachary Peter Gallen is an American profession...,"(39.845416, -75.021701)"
2,Arizona Diamondbacks,Ryne Nelson,https://en.wikipedia.org/wiki/Ryne_Nelson,"Henderson, Nevada","https://en.wikipedia.org/wiki/Henderson,_Nevada",None,Ryne Tanner Nelson is an American professional...,"(36.0292, -115.0253)"
3,Arizona Diamondbacks,Brandon Pfaadt,https://en.wikipedia.org/wiki/Brandon_Pfaadt,"Louisville, Kentucky","https://en.wikipedia.org/wiki/Louisville,_Kent...",None,Brandon Connor Pfaadt is an American professio...,"(38.25611111, -85.75138889)"
4,Arizona Diamondbacks,Humberto Castellanos,https://en.wikipedia.org/wiki/Humberto_Castell...,Tepatitlán,https://en.wikipedia.org/wiki/Tepatitl%C3%A1n,https://upload.wikimedia.org/wikipedia/commons...,Humberto Castellanos is a Mexican professional...,"(20.81388889, -102.76388889)"


In [7]:
# Add the Google Tag Manager and AdSense scripts to the map
analytics = """
<!-- Google tag (gtag.js) -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-2RB0G9NLE8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-2RB0G9NLE8');
</script>

<script async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js?client=ca-pub-4771147286272001"
     crossorigin="anonymous"></script>
"""

In [8]:
# add null image
df['photo_link'].fillna('https://upload.wikimedia.org/wikipedia/commons/e/eb/Baseball_cap.jpg', inplace=True)


C:\Users\efrie\AppData\Local\Temp\ipykernel_19512\2647646414.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['photo_link'].fillna('https://upload.wikimedia.org/wikipedia/commons/e/eb/Baseball_cap.jpg', inplace=True)


In [9]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, LocateControl
from branca.element import Element

# Define MLB team names and colors
mlb_colors = {
    'Team': [
        'Arizona Diamondbacks', 'Atlanta Braves', 'Baltimore Orioles', 'Boston Red Sox',
        'Chicago White Sox', 'Chicago Cubs', 'Cincinnati Reds', 'Cleveland Guardians',
        'Colorado Rockies', 'Detroit Tigers', 'Houston Astros', 'Kansas City Royals',
        'Los Angeles Angels', 'Los Angeles Dodgers', 'Miami Marlins', 'Milwaukee Brewers',
        'Minnesota Twins', 'New York Yankees', 'New York Mets', 'Oakland Athletics',
        'Philadelphia Phillies', 'Pittsburgh Pirates', 'San Diego Padres', 'San Francisco Giants',
        'Seattle Mariners', 'St. Louis Cardinals', 'Tampa Bay Rays', 'Texas Rangers',
        'Toronto Blue Jays', 'Washington Nationals'
    ],
    'Color': [
        '#A71930', '#CE1141', '#DF4601', '#BD3039',
        '#000000', '#CC3433', '#C6011F', '#0C2340',
        '#333366', '#0C2340', '#EB6E1F', '#004687',
        '#BA0021', '#005A9C', '#FF6600', '#FFC425',
        '#002B5C', '#003087', '#002D72', '#003831',
        '#E81828', '#FDB827', '#2F241D', '#FD5A1E',
        '#005C5C', '#C41E3A', '#8FBCE6', '#003278',
        '#134A8E', '#14225A'
    ]
}



# Initialize map centered on USA
m = folium.Map(location=[39.8283, -98.5795], zoom_start=4)

# Add LocateControl to the map
LocateControl().add_to(m)

# Create MarkerCluster for baseball players
marker_cluster = MarkerCluster(name="Baseball Players")

# Add markers for each player
for index, row in df.iterrows():
    coords = row['birth_coords']
    team_name = row['Team']

    # Check if coords is None or not
    if coords is not None:
        # Find the corresponding team color
        if team_name in mlb_colors['Team']:
            color_index = mlb_colors['Team'].index(team_name)
            team_color = mlb_colors['Color'][color_index]

            # Create a div icon with colored circle and team initials (all caps)
            team_initials = team_name.split()[0][0].upper() + team_name.split()[1][0].upper()
            icon = folium.DivIcon(html=f'<div style="background-color: {team_color}; border-radius: 50%; '
                                       f'width: 30px; height: 30px; display: flex; justify-content: '
                                       f'center; align-items: center; color: white; font-weight: bold;">'
                                       f'{team_initials}</div>')

            # Format popup content with wider width
            popup_content = f"""
            <div style="width: 300px;">
                <img src='{row['photo_link']}' width='100px'><br>
                <b><big>{row['Name'].upper()}</big></b><br>
                Team: {row['Team']}<br><br>
                {row['summary']}<br><br>
                <a href='{row['Name_Link']}' target='_blank'>Wikipedia Link</a>
            </div>
            """

            # Create marker and add to MarkerCluster
            marker = folium.Marker([coords[0], coords[1]], popup=popup_content, icon=icon)
            marker.add_to(marker_cluster)

# Add MarkerCluster to the map
marker_cluster.add_to(m)

# Add the Google Analytics tracking code to the map
element = Element(analytics)
m.get_root().html.add_child(element)

# Save the map as HTML file
path = 'maps'
map_filename = 'map_baseball_players.html'
m.save(f"{path}/{map_filename}")

# Display the map (if in a Jupyter notebook or similar environment)
m
